In [1]:
#imports
import random
import math

from numba import njit, prange # library to decrease computation time for the fitness function
import numpy as np #numpy can make fitness into arrays for faster computation
from multiprocessing import Pool #for parallel processing of the fitness function
import tracemalloc #for comparing memory usage among algorithms

In [2]:
#input for the amount of queens
nr_queens = int(input("What number of queens would you like to play with?"))
if nr_queens < 4:
    print("There are no solutions for the n-queens problem for fewer queens than 4")
print(f"Number of queens: {nr_queens}")

Number of queens: 30


In [3]:
#initialisation with random queens

def random_q(nr_queens):
    queens = list(range(nr_queens))  # rows 0 to n-1
    random.shuffle(queens)
    return queens

queens = random_q(nr_queens)
print(queens)

[8, 27, 0, 29, 16, 28, 11, 22, 17, 26, 14, 6, 15, 3, 13, 12, 21, 20, 9, 25, 4, 1, 2, 10, 5, 7, 24, 19, 23, 18]


In [300]:
#fitness for the DFS
def fitness_dfs(queens):
    attacks = 0
    n = len(queens)
    
    for i in range(n):
        for j in range(i + 1, n):
            #check if attacks on the same row — only needed if queens aren't guaranteed unique
            if queens[i] == queens[j]:
                attacks += 1
            #check for attacks on the diagonal
            if abs(queens[i] - queens[j]) == abs(i - j):
                attacks += 1
    return attacks

print(fitness_dfs(queens))

30


In [4]:
#fitness for all other functions

@njit

#for only diagonal moves
#rewriting to arrays for even faster computation

def fitness(queens):
    n = len(queens)
    diag1 = np.zeros(2 * n, dtype=np.int32)
    diag2 = np.zeros(2 * n, dtype=np.int32)

    for col in range(n):
        r = queens[col]
        d1 = col + r
        d2 = col - r + n  #offset to keep index non-negative

        diag1[d1] += 1
        diag2[d2] += 1

    attacks = 0
    for count in diag1:
        if count > 1:
            attacks += count * (count - 1) // 2
    for count in diag2:
        if count > 1:
            attacks += count * (count - 1) // 2

    return attacks

print(fitness(queens))

20


In [301]:
#DFS
tracemalloc.start()

queens_dfs = []
results = int(input("How many solutions do you want?"))

solutions = []

def dfs(row):
    if row == nr_queens:
        solutions.append(queens_dfs.copy())
        return
    
    for i in range(1, nr_queens + 1):
        queens_dfs.append(i)
        if fitness_dfs(queens_dfs) == 0:
            dfs(row + 1)
            if len(solutions) >= results:
                return
        queens_dfs.pop()



dfs(0)
print(solutions)

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6:.2f} MB")

tracemalloc.stop()    


KeyboardInterrupt: 

In [297]:
#hill climbing

queens = random_q(nr_queens)

tracemalloc.start()

loop = 0
restarts = 0

x = True

while x:
    old_fitness = fitness(queens)

    pos1 = random.randint(0, len(queens) - 1)
    pos2 = random.randint(0, len(queens) - 1)
    while pos2 == pos1:
        pos2 = random.randint(0, len(queens) - 1)

    queens[pos1], queens[pos2] = queens[pos2], queens[pos1]  #swap rows

    fit = fitness(queens)

    #revert the swap if fitness didn't improve
    if fit > old_fitness:
        queens[pos1], queens[pos2] = queens[pos2], queens[pos1]

    if (fit == 0):
         x = False
    elif (loop > 1998): #after 2000 iterations, restart the hill climbing algorithm
        queens = random_q(nr_queens)
        loop = 0
        restarts += 1
        if restarts > 200:
            x = False

    loop += 1

print(loop)
print(restarts)

attacks = fitness(queens)

if attacks > 0:
    print(f"No solution found after 200 tries, best solution: {queens}, with {attacks} attacks")
else:
    print(queens)

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6:.2f} MB")

tracemalloc.stop()  

1
201
No solution found after 200 tries, best solution: [76, 102, 49, 199, 163, 144, 171, 178, 99, 19, 23, 140, 176, 133, 42, 167, 145, 191, 93, 136, 180, 189, 117, 158, 37, 111, 161, 172, 1, 141, 45, 146, 187, 13, 66, 143, 177, 149, 174, 77, 157, 41, 122, 33, 100, 183, 80, 84, 195, 56, 65, 90, 17, 35, 70, 125, 20, 47, 192, 110, 38, 15, 96, 83, 153, 139, 48, 175, 169, 94, 120, 151, 91, 31, 103, 89, 71, 86, 121, 24, 182, 131, 29, 25, 7, 16, 78, 67, 52, 40, 11, 168, 60, 181, 85, 8, 137, 108, 119, 152, 59, 184, 36, 46, 10, 124, 179, 160, 82, 26, 101, 155, 44, 166, 2, 32, 12, 128, 107, 39, 64, 73, 27, 14, 113, 3, 135, 18, 58, 114, 105, 87, 106, 193, 170, 30, 0, 147, 186, 104, 126, 109, 194, 196, 142, 88, 63, 132, 198, 9, 69, 53, 138, 62, 81, 22, 112, 127, 150, 185, 92, 197, 97, 28, 156, 55, 34, 50, 134, 68, 98, 118, 159, 123, 54, 51, 6, 130, 74, 5, 188, 129, 95, 173, 4, 75, 72, 61, 162, 57, 190, 116, 79, 154, 43, 115, 165, 148, 164, 21], with 130 attacks
Peak memory usage: 1.49 MB
1
201
No

KeyboardInterrupt: 

In [259]:
#simulated annealing - MOVED TO new_sa.py FILE (DO NOT USE THIS ONE)
tracemalloc.start()

loop = 0
restarts = 0
no_change = 0

# initialize board
queens = random_q(nr_queens)
best_queens = queens[:]
best_fit = fitness(queens)

temp = 1000 if nr_queens < 100 else 2000
x = True

while x:
    old_fitness = fitness(queens)

    #swap 2 queens (1 local move)
    pos1 = random.randint(0, len(queens) - 1)
    pos2 = random.randint(0, len(queens) - 1)
    while pos2 == pos1:
        pos2 = random.randint(0, len(queens) - 1)

    # Swap the rows (rows = values in the list)
    queens[pos1], queens[pos2] = queens[pos2], queens[pos1]

    new_fit = fitness(queens)
    delta = new_fit - old_fitness

    if delta <= 0:
        pass  # accept better or equal move
    else:
        prob = math.exp(-delta / temp)
        if random.random() >= prob:
            # reject move, revert swap
            queens[pos1], queens[pos2] = queens[pos2], queens[pos1]
            
    # update best solution

    if new_fit < best_fit:
        best_fit = new_fit
        best_queens = queens[:]
        no_change = 0
    else:
        no_change += 1

    # success
    if new_fit == 0:
        x = False

    # restart if stuck
    if no_change > 500:
        queens = random_q(nr_queens)
        temp = 1000
        restarts += 1
        no_change = 0
        if restarts > 200 if nr_queens < 100 else 500:
            x = False

    loop += 1
    temp *= 0.995 if nr_queens < 100 else 0.99

print(loop)
print(f"Restarts: {restarts}")

attacks = fitness(best_queens)
if attacks > 0:
    print(f"No solution found, best: {best_queens}, with {attacks} attacks")
else:
    print(f"Solution: {best_queens}")

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6:.2f} MB")

tracemalloc.stop()

KeyboardInterrupt: 

In [5]:
#even more sped up fitness function

@njit(parallel=True)
def batch_fitness(pop):
    n = len(pop)
    out = np.zeros(n, dtype=np.int32)
    for i in prange(n):
        out[i] = fitness(pop[i])
    return out

#print(batch_fitness(queens))

In [7]:
#genetic algorithm - MOVED TO new_ga.py FILE (use this one for n < 100)


tracemalloc.start()


pop_size = int(nr_queens * math.log2(nr_queens) * 2)
max_gen = 5000 #int(nr_queens * math.log2(nr_queens) * 50)


gen = []


#create a generation of random placements


for _ in range(pop_size):
   gen.append(random_q(nr_queens))


#calculate the fitness for each arrangement of queens
def check_fit(gen):
   with Pool(processes=4) as pool:
       fit_list = pool.map(fitness, gen)
   avg = round(sum(fit_list) / len(fit_list), 2)
   return fit_list, avg


def tournament_selection(gen, fit_list, k=3):
   participants = random.sample(list(zip(gen, fit_list)), k)
   #return the gen with the best fitness
   return min(participants, key=lambda x: x[1])[0]


def crossover(gen, fit_list, its):


   new_gen = []


   #create weights based on fitness
   total = sum(1 / (1 + f) for f in fit_list)


   #put the fittest 20% straight into the next generation
   top_int = int(len(fit_list) * 0.2)
   # get indices sorted by fitness (lowest fitness first)
   sorted_indices = sorted(range(len(fit_list)), key=lambda i: fit_list[i])
   # add top 20% to new generation
   for i in sorted_indices[:top_int]:
       new_gen.append(gen[i])


   #every 20th generation, introduce a random combination straight into the next generation
   if its % 20 == 0:
       for _ in range(2):
           new_gen.append(random.sample(range(1, nr_queens + 1), nr_queens)) #only valid permutations


   children = []


   #crossover
   while len(new_gen) < len(gen):
       #p1, p2 = random.choices(gen, weights=weights, k=2)
       p1 = tournament_selection(gen, fit_list, k=3)
       p2 = tournament_selection(gen, fit_list, k=3)


       size = len(p1)


       #perform the crossover: ORDER OX1 Crossover


       #the range (from start to end) where the crossover will happen
       start = random.randint(0, size - 2)
       end = random.randint(start + 1, size - 1)


       child = [None] * size #empty child initialisation
       child[start:end+1] = p1[start:end+1] #fill the child with first parent


       #if the gene isn't already in child, fill it with the other parent genes
       pos = (end + 1) % size
       for gene in p2:
           if gene not in child:
               child[pos] = gene
               pos = (pos + 1) % size #with wrap-around


       mutate(child, its)


       children.append(child)
       new_gen.append(child)


   return new_gen


def mutate(child, its):
   #start at 5% and increase the mutation rate every 1000th gen
   mut_rate = 0.05 + (its // 1000) * 0.05
   #maximum 20% mutation
   mut_rate = min(mut_rate, 0.2)


   if random.random() >= mut_rate:  #95% no mutation, early exit
       return
   else:
       pos1 = random.randint(0, len(child) - 1)
       pos2 = random.randint(0, len(child) - 1)


       #swap 2 columns
       child[pos1], child[pos2] = child[pos2], child[pos1]




def check_sol(fit_list):
   for idx, i in enumerate(fit_list):
       if i == 0:
         return True, idx
   return False, -1




fit_list = batch_fitness(np.array(gen))
avg = round(np.mean(fit_list), 2)




#check fitnesses and create new generation in a loop
x = False
its = 0
while not x and its < max_gen:
   gen = crossover(gen, fit_list, its)
   fit_list = batch_fitness(np.array(gen))
   avg = round(np.mean(fit_list), 2)
   x, idx = check_sol(fit_list)
   if its % 1000 == 0 and its > 0:
       print(f"{its} iterations")
   its += 1




if its >= max_gen:
   print(f"No solutions found after {its} iterations")
else:
   print(fit_list)
   print(f"Population size: {pop_size}")
   print(f"Solution found at index: {idx}")
   print(f"Solution: {gen[idx]}")
   print(f"Number of generations: {its}")


current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6:.2f} MB")


tracemalloc.stop() 


#works up to 100

1000 iterations
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 11 10  5  1 18 14  7  9 12 17  7  1 15  8
  8  1  1 14 19  1 15 21  6  5  8 18  1  5  1 17 12  1  0 10 16 14 10  7
  1 11  9 21 11  7  8  2  8 11  8 10 12  5  5  9 12 13  4  9 19 15 11 15
 19 15 10  8  8  3  8 14 13  5  9 11  9  9  3 14  7 15  4 11 16 15 15  7
 17  4  9 15  7 18 14  1  8 14 14  1  1  8  8 11 18  8  8  1  8 12  9 13
  9  5 10  1  8 16 15 13 11  8 13 12  6 10  6 15 10 12 18 15  1 15  6 12
 12 10 12  3 14 11 15 13  5  8 19 13 12 14 11 17  5  7 16  7 14 11 10 15
 10 12 10  2 11 14 10 11 11 22  9 16  6 13 14 21 14 10  7 12  8  6  9  9
 11  9 17  4  1 16  3 12  6 22 18 11  8  7 17  1 17  7 11 17 10 15  8 15
 16 11  9 10 10 15  8 15 17 15  9  9 13 12  7 11 11 20 10  7  5  1 11  9
  5 14 17 21 12 11]
Population size: 294
Solution found at index: 90
Solution: [29, 21, 17, 5, 7, 12, 18, 24